In [1]:
from pylate import indexes, models, retrieve

model = models.ColBERT(
    model_name_or_path="shresht8/modernBERT_text_similarity_finetune",
)

index = indexes.Voyager(
    index_folder="pylate-index",
    index_name="index",
    override=True,
)

retriever = retrieve.ColBERT(index=index)

c:\Users\shres\anaconda3\envs\rag_case_study\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\shres\anaconda3\envs\rag_case_study\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shres\.cache\huggingface\hub\models--shresht8--modernBERT_text_similarity_finetune. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an

In [2]:
documents_ids = ["1", "2", "3"]

documents = [
    "I have had a very good day", "Mazda 3 is a reliable car", "Since chunk size much smaller compared to the maximum context length of the tokenizer there is scope for increase in chunk length to optimise it for retrieval"
]

# Encode the documents
documents_embeddings = model.encode(
    documents,
    batch_size=32,
    is_query=False, # Encoding documents
    show_progress_bar=True,
)

# Add the documents ids and embeddings to the Voyager index
index.add_documents(
    documents_ids=documents_ids,
    documents_embeddings=documents_embeddings,
)

Adding documents to the index (bs=2000): 100%|██████████| 1/1 [00:00<00:00, 498.61it/s]


In [5]:
documents_embeddings[0].shape

(10, 128)

In [6]:
queries_embeddings = model.encode(
    ["she smirked at the camera while she was driving", "She flies all the time", "what the the best way to divide docs to create embeddings"],
    batch_size=32,
    is_query=True, # Encoding queries
    show_progress_bar=True,
)

scores = retriever.retrieve(
    queries_embeddings=queries_embeddings, 
    k=10,
)

print(scores)

Retrieving documents (bs=50): 100%|██████████| 1/1 [00:01<00:00,  1.71s/it]

[[{'id': '1', 'score': 30.47486114501953}, {'id': '3', 'score': 30.46074676513672}, {'id': '2', 'score': 30.432344436645508}], [{'id': '1', 'score': 30.613506317138672}, {'id': '3', 'score': 30.583486557006836}, {'id': '2', 'score': 30.569419860839844}], [{'id': '3', 'score': 30.844350814819336}, {'id': '1', 'score': 30.753984451293945}, {'id': '2', 'score': 30.70146369934082}]]
